In [19]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,sum,round,dense_rank
from pyspark.sql.window import Window


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

In [2]:
ParkingTransactions = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "ParkingTransactions") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [3]:
ParkingTransactions.show()

+------+------+-------------------+-------------------+--------+
|lot_id|car_id|         entry_time|          exit_time|fee_paid|
+------+------+-------------------+-------------------+--------+
|     1|  1001|2023-06-01 08:00:00|2023-06-01 10:30:00|    5.00|
|     1|  1001|2023-06-02 11:00:00|2023-06-02 12:45:00|    3.00|
|     2|  1001|2023-06-01 10:45:00|2023-06-01 12:00:00|    6.00|
|     2|  1002|2023-06-01 09:00:00|2023-06-01 11:30:00|    4.00|
|     3|  1001|2023-06-03 07:00:00|2023-06-03 09:00:00|    4.00|
|     3|  1002|2023-06-02 12:00:00|2023-06-02 14:00:00|    2.00|
+------+------+-------------------+-------------------+--------+



In [28]:
pk_acg_df = ParkingTransactions\
    .withColumn("time_diff",(col("exit_time").cast("long")-col("entry_time").cast("long"))/3600)\
    .groupby(col("car_id"))\
    .agg(sum("fee_paid").alias("total_fee_paid"),
         sum("time_diff").alias("total_time_diff"))\
    .withColumn("avg_hourly_fee",round(col("total_fee_paid")/col("total_time_diff"),2))\
    .drop("total_time_diff")


most_pk_df = ParkingTransactions\
    .withColumn("time_diff",(col("exit_time").cast("long")-col("entry_time").cast("long"))/3600)\
    .groupby(col("car_id"),col("lot_id"))\
    .agg(sum("time_diff").alias("total_time_diff"))\
    .withColumn("rnk",dense_rank().over(Window.partitionBy("car_id").orderBy(col("total_time_diff").desc())))\
    .filter(col("rnk")==1)\
    .select("car_id","lot_id")\
    .withColumnRenamed("car_id","c_id")

pk_acg_df\
    .join(most_pk_df,pk_acg_df.car_id == most_pk_df.c_id,"inner")\
    .select("car_id","total_fee_paid","avg_hourly_fee","lot_id")\
    .show()



+------+--------------+--------------+------+
|car_id|total_fee_paid|avg_hourly_fee|lot_id|
+------+--------------+--------------+------+
|  1001|         18.00|           2.4|     1|
|  1002|          6.00|          1.33|     2|
+------+--------------+--------------+------+

